##  Importing Dependencies

In [1]:
import sklearn
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgbm
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


## Defining Functions for Future Use

Using the quartiles to remove possible outliers

In [2]:
# image explaining how quartiles can be used for outlier removal has been added in gthub
# One might use other methods too for outlier removal
def outlier_threshold(dataframe, variable):
    Q1 = dataframe[variable].quantile(0.01)
    Q3 = dataframe[variable].quantile(0.99)
    IQR = Q3 - Q1
    up_limit = Q3 + 1.5 * IQR
    low_limit = Q1 - 1.5 * IQR
    return low_limit, up_limit

def replace_with_threshold(dataframe, variable):
    low_limit, up_limit = outlier_threshold(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    

# Column dropped
def feature_cols(df) :
    cols = [c for c in df.columns if c not in ['row_id', 'time_id', 'date_id']]
    df = df[cols]    
    return df

# Preporcessing
def pre_process(df):
    df = pd.get_dummies(df, columns=['imbalance_buy_sell_flag'], prefix='imbalance_flag', drop_first=True)
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    return df

In [3]:
def engineered_features(df):
    # lists of prices and sizes defined separately
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    # total volume of the stock is ask size+ bid size
    df["volume"] = df.eval("ask_size + bid_size")
    
    # mid price of the stock (on avg)
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    
    # Volume/Size Features
    df["bid_ask_size_ratio"] = df["bid_size"] / df["ask_size"]
    df["imbalance_bid_size_ratio"] = df["imbalance_size"] / df["bid_size"]
    df["imbalance_ask_size_ratio"] = df["imbalance_size"] / df["ask_size"]
    df["matched_size_ratio"] = df["matched_size"] / (df["bid_size"] + df["ask_size"])
    
    df["ref_wap_difference"] = df["reference_price"] - df["wap"]
    df["bid_ask_spread"] = df["ask_price"] - df["bid_price"]
    df["near_far_price_difference"] = df["far_price"] - df["near_price"]
    
    # Trend Indicators
    # according to the Dataset Description, all price related columns are converted to a price move 
    # relative to the stock wap at the beginning of the auction period.
    df["wap_rate_of_change"] = df.groupby('stock_id')["wap"].pct_change()
    df["wap_momentum"] = df.groupby('stock_id')["wap"].diff()
    
    df["auction_start"] = (df["seconds_in_bucket"] == 0).astype(int)
    df["auction_end"] = (df["seconds_in_bucket"] == 550).astype(int)
    df["time_since_last_change"] = df.groupby('stock_id')['imbalance_buy_sell_flag'].diff(periods=1).ne(0).cumsum()
    df["time_until_auction_close"] = 600 - df["seconds_in_bucket"]
      
    # liquidity imbalance is a measure of the disparity between the quantity of buy orders and sell orders
    # 1, it indicates a higher proportion of buy orders (bids), suggesting a potential buying pressure
    #-1, it indicates a higher proportion of sell orders (asks), suggesting a potential selling pressure
    # 0, it suggests a more balanced distribution of buy and sell orders.
    # refer to the image in my github to understand more
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    
    # A measure of the remaining order imbalance after a certain amount has been matched or executed.
    # If the result is positive, it indicates that there is still a remaining order imbalance biased towards buying.
    # If the result is negative, it indicates that there is still a remaining order imbalance biased towards selling.
    # If the result is close to zero, it suggests a more balanced distribution between buy and sell orders.
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
   
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])

    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)

    #lagged values
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)


   # the definations and the correlations of the terms used can be found in github         
            
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    df['wap_askprice_diff'] = df['ask_price'] - df['wap'] 
    df['wap_bidprice_diff'] = df['wap'] - df['bid_price']
    df['wap_askprice_diff_urg'] = df['wap_askprice_diff'] * df['liquidity_imbalance'] 
    df['wap_bidprice_diff_urg'] = df['wap_bidprice_diff'] * df['liquidity_imbalance']
    df['bid_size_ask_size_diff'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imbalance_size_matched_size_diff'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
# Function to generate time and stock-related features
# According to the Dataset Description, target is the 60 second future move in the wap of the stock, 
# less the 60 second future move of the synthetic index.

    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  # Seconds
    df["minute"] = df["seconds_in_bucket"] // 60  # Minutes
       
    global_stock_id_feats = {
        "median_size": df.groupby("stock_id")["bid_size"].median() + df.groupby("stock_id")["ask_size"].median(),
        "std_size": df.groupby("stock_id")["bid_size"].std() + df.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df.groupby("stock_id")["bid_size"].max() - df.groupby("stock_id")["bid_size"].min(),
        "median_price": df.groupby("stock_id")["bid_price"].median() + df.groupby("stock_id")["ask_price"].median(),
        "std_price": df.groupby("stock_id")["bid_price"].std() + df.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df.groupby("stock_id")["bid_price"].max() - df.groupby("stock_id")["ask_price"].min(),
    }
    
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())
        
    return df.replace([np.inf, -np.inf], 0)

## Reading and Understanding The Data

In [4]:
df_ = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv") # training data
test = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv") # testing data

Make a copy of 'df' for working purpose 

In [5]:
df = df_.copy()

### Understandings and Findings
1. row_id is a string concatenation of date_id, seconds_in_bucket, and stock_id, separated with underscores.
2. target is the target variable we want to predict, for its namesake.
3. 200 * 481 * 55 = 5,291,000, which is roughly equal to the number of total rows. We assume that the training data consist of 200 * 481 = 96,200 time series; Each time series are 55 steps long, and each time series represents the last 10 mins of a given stock on a given trading date. 5,291,000 - 5,237,980 = 53,020, so there are some data missing.
4. There are some stocks missing data on some days entirely.

Rest of the findings and resources can be accessed from github

In [6]:
df.isnull().sum().sum()

5752930

# PreProcessing

Checking for null values

In [7]:
# One day's data was missing for four stocks
askpricena = df[df['ask_price'].isna()]
print(askpricena.stock_id.unique())
print(askpricena.date_id.unique())
print(askpricena.seconds_in_bucket.unique())
for date in askpricena.date_id.unique():
    x = df[df.date_id == date]
    print(date, x[x.ask_price.isna()]['stock_id'].unique().tolist())

[131 101 158  19]
[ 35 328 388 438]
[  0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170
 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350
 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530
 540]
35 [131]
328 [101]
388 [158]
438 [19]


In [8]:
# I'm throwing it away for now to see where the other missing data is.
df = df.dropna(subset=["ask_price"], axis=0)
df.loc[df['seconds_in_bucket'] <= 300, "near_price"] = 0
df.loc[df['seconds_in_bucket'] <= 300, "far_price"] = 0
df['far_price'] = df['far_price'].interpolate()

Null values have been now taken care of

In [9]:
df.isnull().sum().sum()

0

The engineered features can be accessed from the function I defined above

In [10]:
df = engineered_features(df)

# Modelling of model

I would be using a simple lightGBM workflow as baseline

In [11]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error

In [12]:
# dropping off the target column as its the dependent variable
X = df.drop(["target", "row_id"], axis=1)
y = df[["target"]]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44)

In [13]:
lgb_model = lgbm.LGBMRegressor()
lgb_model.fit(X, y) # didn't split so as to take more input into consideration

LGBMRegressor()

In [14]:
# pred = lgb_model.predict(X)

In [15]:
#import seaborn as sns

# Visualization

In [16]:
def plot_importance(model, features, num=len(X)):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show(block=True)

In [17]:
# plot_importance(lgb_model, X)

# Tuning of model

In [18]:
"""lgb_params = {"learning_rate": [0.01, 0.1],
               "n_estimators": [100, 300, 500, 1000],
               "colsample_bytree": [0.5, 0.7, 1]}"""

'lgb_params = {"learning_rate": [0.01, 0.1],\n               "n_estimators": [100, 300, 500, 1000],\n               "colsample_bytree": [0.5, 0.7, 1]}'

In [19]:
"""lgbm_best_grid = GridSearchCV(lgb_model, lgb_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)"""

'lgbm_best_grid = GridSearchCV(lgb_model, lgb_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)'

In [20]:
"""lgb_final = lgb_model.set_params(**lgbm_best_grid.best_params_, random_state=17).fit(X, y)"""

'lgb_final = lgb_model.set_params(**lgbm_best_grid.best_params_, random_state=17).fit(X, y)'

# Submission

In [21]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [22]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    test = engineered_features(test)
    test_df = test.drop(["row_id"], axis=1)
    sample_prediction['target'] = lgb_model.predict(test_df)
    env.predict(sample_prediction)
    
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


I would now try to improve the baseline model, by tuning its parameters and adding more visualization to understand the underlying complex terminologies involved in the problem statement.